# Crawler Noticias



## Introdução

Estudo projeto de construção de um bot crawler para extrair e indexar noticias de sites.

É recomendavel baixar uma única vez o html do site para reduzir a necessidade de acessar o site várias vezes para cada processamento reduzindo tempo e custo computacionais para máquina local e servidor.

Usamos módulo _Requests_ para baixar o html da página a ser processada e com módulo _BeautifulSoup_ processamos o conteúdo html buscando por conteúdos especificos. Ao inspecionar o código HTML verificamos em quais blocos estão as principais noticias - em geral em tags como "section", "h1"/"h2"/"h3" e "a" - considerando o seletor css que especifica cada bloco de interesse afim de refinar a busca e garantir que serão extraindos os trechos relevantes.

Sites com boas práticas de HTML/CSS tendem a serem mais faceis de serem processados e indexados por mecanismos de buscas facilitando os usuários em encontrar seus conteúdos em pesquisas aumentando assim o engajamento em suas plataformas contribuindo para seu melhor desenvolvimento.

Neste presente momento o projeto está considerando três portais de notícias que são _BBC Brasil_, _CNN Brasil_ e _G1/Globo_. 

Créditos e direitos reservados às referentes plataformas mencionadas.

(Obs.: Conteúdo livre com fins informativos e de divulgação. )


## Módulos

Abaixo _import_ dos módulos que contém os recursos usados.

* BeautifulSoup
* Requests
* re (regular expression)
* crawlernewsg1 (acessa site e extrai lista de principais noticias na página principal do site)
* IPython.core.display -> display, HTML (renderiza conteúdos html )

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from crawlernewsg1 import *
import random

# Render html content
from IPython.core.display import display, HTML

## Modelagem Portal G1



Capturando noticias do portal G1 da Globo
Funções básicas. Especifincando "selector css" classe 'bastian-page'  para tags div.

In [4]:
# Baixando html do portal de noticia para processamento e salvando em disco.
url = 'https://g1.globo.com'
attr = {'class': 'bastian-page'}
data = g1_(requests.get( url ).content, 'div', attr, debug=False)
dw = requests.get('https://g1.globo.com/').content
with open('/tmp/data', 'w') as fl:
    fl.write( dw.decode() )
    

In [5]:
type(data)

tuple

In [6]:
# Replace field name 'titulo' for 'title' name.
#dw = [ { key.replace('titulo', 'title'):value for key, value in i.items() } for i in dw ]
#dw = [ { key.replace('url', 'href'):value for key, value in i.items() } for i in dw ]
#replKey = lambda listdc, old_key, new_key: [ ]
def replKey(listdc, old_key, new_key  ):
    dt = []
    
    
    for i in listdc:
        for keys, value in i.items():
            dt.append( { keys.replace('url', 'href'): value } ) 
            
    return dt

data = replKey(data[0], 'url', 'href' )

print(data)

[{'title': "VÍDEO: veja como foi a 1ª 'live' de Marte"}, {'href': 'https://g1.globo.com/ciencia/noticia/2023/06/02/veja-como-foi-a-primeira-live-de-marte-feita-pela-agencia-espacial-europeia.ghtml'}, {'title': 'Jeff Machado foi morto com fio de telefone, diz suspeito preso'}, {'href': 'https://g1.globo.com/rj/rio-de-janeiro/noticia/2023/06/02/garoto-de-programa-acusa-amigo-pela-morte-do-ator-jeff-e-confessa-que-vitima-foi-morta-com-fio-de-telefone-apos-ter-sido-dopado.ghtml'}, {'title': "Prisão no caso Jeff Machado, 'zona da morte' do Everest e mais vídeos"}, {'href': 'https://g1.globo.com/playlist/videos-para-assistir-agora.ghtml'}, {'title': "Dino visitou Lira após operação da PF: 'Falei o óbvio'"}, {'href': 'https://g1.globo.com/politica/blog/andreia-sadi/post/2023/06/02/dino-e-lira-se-reunem-apos-operacao-policial-ordem-judicial-explica-ministro.ghtml'}, {'title': "'Tem que falar com quem não gosta da gente', diz Lula sobre Congresso"}, {'href': 'https://g1.globo.com/politica/notic

In [7]:
dw, cwn = g1_( dw, 'div', attr)

Ao abrir link da noticia pesquisar pela tag *'p'* com atributos **class** com valor **"content-text__container** definir o tamanho para caso extrair só parte do corpo do texto.


Retornando uma lista dos itens encontrados (como visto acima no código) pegamos estes itens e os concatenamos exibintido o texto no corpo da noticia. (_Como visto abaixo_)

Cada "_evt" (_css selector_ class) class css em "bastian-page" refere-se a uma noticia na lista central de noticias.
Dentro de cada "_evt" haverá "bastian-feed-item" e neste o feed-post. 

**feed-post-body** _contêm_  ( 'feed-post-link', 'feed-post-body-title', 'feed-post-body-resumo')

**bastian-feed-item** _contem_ um feed-post-body referindo-se a cada item (noticia)


Para link da noticia (quando acessando a noticia)

**content-head__title** em tag 'h1' (Título da noticia)

**content-head__subtitle** em tag 'h2' (subtitulo/resumo da noticia)

**content-text__container** corpo do texto da noticia css-selector, tag 'p' (pegar só a primeira referente ao primeiro paragrafo da noticia)

In [8]:
news2 = '<h1 style="padding: 12px;">Notícias</h1>'
news2 += '<br><br>'.join( [ '<br>'.join( [ str( i['title'] ) , str( i[ 'url' ]  ) ] ) for i in dw ] )
display( HTML( news2 ))

## CNN Crawler de Noticias do Portal

In [9]:
cnn_data = requests.get('https://www.cnnbrasil.com.br/')


In [10]:
cnn_soup = BeautifulSoup(cnn_data.content, 'html.parser')
cnn_nw_data = cnn_soup.find_all('section')
cnn_nw_data[0]

<section class="hot__content"> <div class="carousel"> <div class="carousel__screen infinite"> <ul class="carousel__track"> <li class="carousel__item"><div class="hot__list__itens"> <a href="https://www.cnnbrasil.com.br/economia/incentivo-a-compra-de-carros-vai-durar-quatro-meses-e-custar-r-500-milhoes/" target="_self" title="DANIEL RITTNER Incentivo à compra de carros vai durar quatro meses e custar R$ 500 milhões"> <img class="tb" src="https://www.cnnbrasil.com.br/wp-content/uploads/sites/12/Reuters_Direct_Media/BrazilOnlineReportBusinessNews/tagreuters.com2023binary_LYNXMPEJ330JV-FILEDIMAGE.jpg?w=65&amp;h=37&amp;crop=1" title="DANIEL RITTNER Incentivo à compra de carros vai durar quatro meses e custar R$ 500 milhões"/> <div class="i_ch"> <i></i> <span class="ch"> DANIEL RITTNER </span> </div> <span class="tp"> Incentivo à compra de carros vai durar quatro meses e custar R$ 500 milhões </span> </a></div></li><li class="carousel__item"><div class="hot__list__itens"> <a href="https://ww

In [11]:
c = cnn_nw_data[0].find('a')
#dir(c)


In [12]:
c.attrs

{'href': 'https://www.cnnbrasil.com.br/economia/incentivo-a-compra-de-carros-vai-durar-quatro-meses-e-custar-r-500-milhoes/',
 'title': 'DANIEL RITTNER Incentivo à compra de carros vai durar quatro meses e custar R$ 500 milhões',
 'target': '_self'}

In [13]:
cnn_list_news = []
aux = None 
for news in cnn_nw_data:
    aux = news.find('a')
    
    
    try :
        
        aux = aux.attrs
        cnn_list_news.extend( [ { 'title': aux['title'], 'href': aux['href'] } ] ) 

    except:
        pass

print(cnn_list_news)

[{'title': 'DANIEL RITTNER Incentivo à compra de carros vai durar quatro meses e custar R$ 500 milhões', 'href': 'https://www.cnnbrasil.com.br/economia/incentivo-a-compra-de-carros-vai-durar-quatro-meses-e-custar-r-500-milhoes/'}, {'title': 'Lula sobre aprovação de MP: "Você tem que conversar com quem não gosta da gente"', 'href': 'https://www.cnnbrasil.com.br/politica/lula-diz-que-aprovacao-de-mp-correu-risco-e-que-e-preciso-conversar-com-quem-nao-votou-na-gente/'}, {'title': 'Agência Europeia faz primeira transmissão ao vivo da história de Marte; veja como foi', 'href': 'https://www.cnnbrasil.com.br/tecnologia/esa-faz-primeira-transmissao-ao-vivo-da-historia-de-marte/'}, {'title': 'Política', 'href': 'https://www.cnnbrasil.com.br/politica/'}, {'title': 'Pop', 'href': 'https://www.cnnbrasil.com.br/pop/'}, {'title': 'Mercado', 'href': '/cotacoes/bolsa'}, {'title': 'Economia', 'href': 'https://www.cnnbrasil.com.br/economia/'}, {'title': 'Internacional', 'href': 'https://www.cnnbrasil.co

In [14]:
cnn_list_news_ma = None
for i in cnn_list_news:
    
    try:
        print(f"\n{i['title']} \n{i['href']}\n\n" ) 
    except:
        pass 
    #keys = i.keys()
    #print(keys)   
    

    #print( f'\t\n\n{out}') 
    #out = '' 



DANIEL RITTNER Incentivo à compra de carros vai durar quatro meses e custar R$ 500 milhões 
https://www.cnnbrasil.com.br/economia/incentivo-a-compra-de-carros-vai-durar-quatro-meses-e-custar-r-500-milhoes/



Lula sobre aprovação de MP: "Você tem que conversar com quem não gosta da gente" 
https://www.cnnbrasil.com.br/politica/lula-diz-que-aprovacao-de-mp-correu-risco-e-que-e-preciso-conversar-com-quem-nao-votou-na-gente/



Agência Europeia faz primeira transmissão ao vivo da história de Marte; veja como foi 
https://www.cnnbrasil.com.br/tecnologia/esa-faz-primeira-transmissao-ao-vivo-da-historia-de-marte/



Política 
https://www.cnnbrasil.com.br/politica/



Pop 
https://www.cnnbrasil.com.br/pop/



Mercado 
/cotacoes/bolsa



Economia 
https://www.cnnbrasil.com.br/economia/



Internacional 
https://www.cnnbrasil.com.br/internacional/



Viagem & Gastronomia 
https://www.cnnbrasil.com.br/viagemegastronomia/



Nacional 
https://www.cnnbrasil.com.br/nacional/



Saúde 
https://www.

In [15]:
len(dw)

8

In [16]:
len(cnn_list_news)

15

## BBC Brasil 

Crawler das notícias do portal do site BBC Brasil.

In [17]:
url_bbc = 'https://www.bbc.com/portuguese'
url_bbc_base = 'https://www.bbc.com'

In [18]:
bbc_data = requests.get(url_bbc)
bbc_soup = BeautifulSoup(bbc_data.content, 'html.parser')
bbc_sections =  bbc_soup.find_all('section' , class_= 'bbc-iinl4t')

In [19]:
bbc_news_lists = []

``` python
url_bbc_base+bbc_sections[0].select('a')[0]['href']
```

Saída 'https://www.bbc.com/portuguese/brasil-63507138'

In [20]:
bbc_sections[0].select('a')[0].text

"'Cabo de guerra' entre Lula e Lira é fruto do 'presidencialismo do descaso' criado por Bolsonaro, diz professor da USP"

In [21]:
for news in bbc_sections:
    bbc_news_lists.extend( [ { 'title': i.text, 'url': i['href'] } for i in news.select('a') ] )

In [22]:
bbc_news_lists = []
aux = None 

for section_news in bbc_sections:
    for news in section_news.select( 'a' ):
        if news['href'].find('topic') != -1:
            pass
        else :
            if news['href'].find('https') != -1:
                bbc_news_lists.extend( [ { 'title': news.text, 'href': news['href'] } ] )
            else:
                bbc_news_lists.extend( [ { 'title': news.text, 'href': url_bbc_base+news['href'] } ] )

In [23]:
print(f'Número de notícias: {len(bbc_news_lists)}')

Número de notícias: 62


In [24]:
random.choices(bbc_news_lists, k = 3)

[{'title': "O medo que ainda reina no 'país mais bombardeado da história'",
  'href': 'https://www.bbc.com/portuguese/articles/c6pd3p5d7dvo'},
 {'title': "Por que críticos dizem que episódio final de Succession é 'perfeito' e 'brutal'",
  'href': 'https://www.bbc.com/portuguese/articles/clj1883r823o'},
 {'title': "Collor: 'impichado', inocentado e, agora, condenado a 8 anos de prisão",
  'href': 'https://www.bbc.com/portuguese/articles/c4npdpdx0zgo'}]

In [25]:
print(url_bbc_base+'/portuguese/topics/cz74k71p8ynt')

https://www.bbc.com/portuguese/topics/cz74k71p8ynt


In [26]:
with open('bbc_news.txt', 'w') as fl:
    for i in bbc_news_lists:
        fl.write(i['title'] +'\n' + i['href'] +'\n\n')
    fl.close()

## Portal Band

Busca de lista de notícias do portal Band.

In [27]:
import requests
from bs4 import BeautifulSoup

In [28]:
url_band = 'https://www.band.uol.com.br/'

In [29]:
band_data = requests.get(url_band).content
soup_band = BeautifulSoup(band_data, 'html.parser')


#### Band list sections 

In [30]:
news_band_section = soup_band.find_all('section')

In [31]:
print(news_band_section[0])
print(f'Número: {len(news_band_section)}')

<section _ngcontent-sc38="" class="menu__section"><span _ngcontent-sc38="">Menu</span><!-- --><!-- --></section>
Número: 17


#### Band home main

![bandcapt.png](bandcapt.png)

1. tag main class -> home
2. Tag section com class=hardnews
3. tag div's conteúdos com para classes hardnews__highlight e cards

In [32]:
import re
# Tag 'main' e class=[home container]. (variável band_main_home)
band_main_home = soup_band.find_all('main', class_='home')
# -----------------------
#dir(band_main_home[0] )
#print(str( band_main_home ) )
soup_band2 = BeautifulSoup( str( band_main_home ) , 'html.parser')

bmainhardnews = soup_band2.find_all('section', 'hardnews')
soup_blocknews = BeautifulSoup( str( bmainhardnews[0] ), 'html.parser' )
bn1 = soup_blocknews.find_all( 'div', class_='hardnews__highlight' )

soup_links_news = BeautifulSoup(str( bn1 ), 'html.parser') 
bn1_link_news = soup_links_news.find_all( 'a'  )#, class_= [ 'link image','related', 'link']  ) 
bn2 = soup_blocknews.find_all( 'div', class_='cards' )

soup2bn2 = BeautifulSoup( str( bn2[0] ), 'html.parser' ) 
bn2_cards_links = soup2bn2.find_all( 'a' )

# Tag: div Class -> hardnews__highlight and card. Duas seções/blocos de noticias.
#print('\n> bn1 ', bn1[0] )
#print('\n> bn2 ', bn2[0] )
print('\n bn1_link_news ', len(bn1_link_news ))
print('\n bn1_link_news ', bn1_link_news[1] )
print('\n'+30*'-')
print('\n bn2_cards_links (size)', len(bn2_cards_links) )
print('\ bn2_cards_links ', bn2_cards_links[-1] ) #Pegando o último elemento da lista e exibindo.



 bn1_link_news  4

 bn1_link_news  <a _ngcontent-sc11="" href="https://www.band.uol.com.br/noticias/se-aprovado-zanin-podera-ficar-28-anos-no-stf-16606869"><i _ngcontent-sc11="" class="custom-icon fas fa-square"></i><!-- -->Se for aprovado, Cristiano Zanin poderá ficar 28 anos no Supremo</a>

------------------------------

 bn2_cards_links (size) 20
\ bn2_cards_links  <a _ngcontent-sc10="" href="https://www.band.uol.com.br/bandnews-fm/noticias/monica-bergamo-aliados-avaliam-que-lira-exagerou-e-termina-semana-derrotado-16606903" title="Aliados avaliam que Lira exagerou e termina semana derrotado"><h2 _ngcontent-sc10="" class="title"><i _ngcontent-sc10="" class="custom-icon fas fa-square"></i><!-- --> Aliados avaliam que Lira exagerou e termina semana derrotado</h2></a>


In [33]:
#print(dir(bn1_link_news[0] ) )
#bn1_link_news[0]['href']

# Lista de links de notícias no bloco {tag: div, class_: 'hardnews__highlight' } do HTML.
for i in bn1_link_news:
    print('\n> ', i.text +'\n | '+i['href'])


>  EXCLUSIVOGilmar sobre Zanin no STF: "Não acho que o presidente deva selecionar inimigos"Em entrevista ao Jornal Gente, ministro defendeu indicação de Lula à Corte
 | https://www.band.uol.com.br/noticias/gilmar-sobre-zanin-no-stf-nao-acho-que-o-presidente-deva-selecionar-inimigos-16606885

>  Se for aprovado, Cristiano Zanin poderá ficar 28 anos no Supremo
 | https://www.band.uol.com.br/noticias/se-aprovado-zanin-podera-ficar-28-anos-no-stf-16606869

>  Zanin deve ser sabatinado no Senado até o final deste mês
 | https://www.band.uol.com.br/noticias/indicado-de-lula-ao-stf-zanin-deve-ser-sabatinado-no-senado-ate-o-final-do-mes-16606932

>  Reinaldo: Moro, Deltan e Ives Gandra contra Zanin? Indicado se qualifica
 | https://www.band.uol.com.br/bandnews-fm/noticias/reinaldo-moro-deltan-e-ives-gandra-contra-zanin-no-stf-indicado-se-qualifica-mais-16606887


In [34]:
for i in bn2_cards_links:
    print('\n> ', i['title'] +'\n | '+i['href'])


>  Verstappen supera Alonso e lidera segundo treino do GP da Espanha
 | https://www.band.uol.com.br/esportes/automobilismo/formula-1/ao-vivo/gp-da-espanha-de-formula-1-treino-livre-2

>  Verstappen supera Alonso e lidera segundo treino do GP da Espanha
 | https://www.band.uol.com.br/esportes/automobilismo/formula-1/ao-vivo/gp-da-espanha-de-formula-1-treino-livre-2

>  Verstappen supera Alonso e lidera segundo treino do GP da Espanha
 | https://www.band.uol.com.br/esportes/automobilismo/formula-1/ao-vivo/gp-da-espanha-de-formula-1-treino-livre-2

>  Mercedes terá atualizações na Espanha após novidades em Mônaco
 | https://www.band.uol.com.br/esportes/automobilismo/formula-1/noticias/gp-da-espanha-atualizacoes-em-monaco-16606579

>  Saiba onde e como assistir aos treinos e à corrida na Espanha
 | https://www.band.uol.com.br/esportes/automobilismo/formula-1/noticias/f1-2023-espanha-16606363

>  Moraes ironiza Galo: "Quem mandou não saber bater pênalti"
 | https://www.band.uol.com.br/mina

#### Função "trackBandNews" (Obtendo notícias do portal da Band)

In [35]:
# Type list of dictionary
ListDict = list[ dict ]

def trackBandNews(data: str) -> ListDict:
    """
    data: str [dados] text html
    return list of news with fields 'Title of news' and 'url' for news.
    l = [
    {'title': 'string', 'href': link},
    ...
    ]
    
    """
    # Lista de notícias
    result = []
    
    # Obtendo conteúdo html
    #band_data = requests.get(url_band).content
    soup_band = BeautifulSoup(data, 'html.parser')
    
    # Primeiro acesso tag main (bloco principal que contêm conteúdos como as noticias principais e seus links.
    # Tag 'main' e class=[home container]. (variável band_main_home)
    band_main_home = soup_band.find_all('main', class_='home')
    
    # Capturando html do bloco maior em main que contêm as noticias.
    soup_band2 = BeautifulSoup( str( band_main_home ) , 'html.parser')
    
    # Este block ( tag:section, class:hardnews) filho da tag main contem dois blocos de noticias a serem tratados insoladamente.
    bmainhardnews = soup_band2.find_all('section', 'hardnews')
    soup_blocknews = BeautifulSoup( str( bmainhardnews[0] ), 'html.parser' )
    bn1 = soup_blocknews.find_all( 'div', class_='hardnews__highlight' )

    
    soup_links_news = BeautifulSoup(str( bn1 ), 'html.parser') 
    bn1_link_news = soup_links_news.find_all( 'a'  )#, class_= [ 'link image','related', 'link']  ) 
    bn2 = soup_blocknews.find_all( 'div', class_='cards' )

    soup2bn2 = BeautifulSoup( str( bn2[0] ), 'html.parser' ) 
    bn2_cards_links = soup2bn2.find_all( 'a' )

    # Tag: div Class -> hardnews__highlight and card. Duas seções/blocos de noticias.
    #print('\n> bn1 ', bn1[0] )
    #print('\n> bn2 ', bn2[0] )
    #print('\n bn1_link_news ', len(bn1_link_news ))
    #print('\n bn1_link_news ', bn1_link_news[1] )
    #print('\n'+30*'-')
    #print('\n bn2_cards_links (size)', len(bn2_cards_links) )
    #print('\ bn2_cards_links ', bn2_cards_links[-1] ) #Pegando o último elemento da lista e exibindo.
    
    # Lista de links de notícias no bloco {tag: div, class_: 'hardnews__highlight' } do HTML.
    for i in bn1_link_news:
        result.append( { 'title': i.text, 'href': i['href'] } )
        #print('\n> ', i.text +'\n | '+i['href'])
        
    for i in bn2_cards_links:
        result.append( { 'title' : i['title'], 'href':  i['href'] } )
        #print('\n> ', i['title'] +'\n | '+i['href'])
    
    return result

In [36]:
band_news = trackBandNews( band_data )
band_news

[{'title': 'EXCLUSIVOGilmar sobre Zanin no STF: "Não acho que o presidente deva selecionar inimigos"Em entrevista ao Jornal Gente, ministro defendeu indicação de Lula à Corte',
  'href': 'https://www.band.uol.com.br/noticias/gilmar-sobre-zanin-no-stf-nao-acho-que-o-presidente-deva-selecionar-inimigos-16606885'},
 {'title': 'Se for aprovado, Cristiano Zanin poderá ficar 28 anos no Supremo',
  'href': 'https://www.band.uol.com.br/noticias/se-aprovado-zanin-podera-ficar-28-anos-no-stf-16606869'},
 {'title': 'Zanin deve ser sabatinado no Senado até o final deste mês',
  'href': 'https://www.band.uol.com.br/noticias/indicado-de-lula-ao-stf-zanin-deve-ser-sabatinado-no-senado-ate-o-final-do-mes-16606932'},
 {'title': 'Reinaldo: Moro, Deltan e Ives Gandra contra Zanin? Indicado se qualifica',
  'href': 'https://www.band.uol.com.br/bandnews-fm/noticias/reinaldo-moro-deltan-e-ives-gandra-contra-zanin-no-stf-indicado-se-qualifica-mais-16606887'},
 {'title': 'Verstappen supera Alonso e lidera seg

#### Percorrer listas de elementos em 'news_band_section'

In [37]:
aux = []
list_ = None
for secs in news_band_section:
    list_ = secs.find_all('a', class_='link')
    for i in list_:
        
        try:
            aux.extend( [ { 'title': i.h3.text+'. '+ i.h2.text, 'href': i['href']} ] )
        except:
            aux.extend( [ { 'title': i.text, 'href': i['href'] } ] )
    #aux.extend([ { 'title': i}  for i in list_ ]  )
news_list_band = [] 
news_list_band.extend(random.choices(aux, k=4 ) )

In [38]:
n = soup_band.find_all('a')

In [39]:
import random

random.choices(news_list_band, k = 4)[0:]
for i in news_list_band:
    print(f"\n{i['title']}\n{i['href']}")



https://www.band.uol.com.br/noticias/bora-brasil/ultimas/policia-prende-um-dos-suspeitos-da-morte-do-ator-jeff-machado-em-operacao-no-rio-16606867

Veja opções de presentes para aquecer o seu amor
https://www.band.uol.com.br/band-shop/guia-de-compras/dia-dos-namorados-presentes-para-aquecer-o-seu-amor-no-inverno-16605832

BandNews FM
/bandnews-fm/noticias/eduardo-bauermann-e-suspenso-por-12-jogos-por-manipulacao-de-resultados-16606835

FUTEBOL
https://www.band.uol.com.br/minas-gerais/noticias/corinthiano-ministro-alexandre-de-moraes-zoa-atletico-16606962


In [40]:
n[0]

<a _ngcontent-sc33="" aria-label="Band" class="logo logo--portal logo-name--Band logo--image center" href="/"><img _ngcontent-sc33="" alt="Band" class="logo-image" height="35" src="https://pubimg.band.uol.com.br/Files/logo-band-2022-v2.png?v2" width="101"/><!-- --><!-- --><!-- --></a>

In [41]:
#print(dir(n[0]))
n[0]

<a _ngcontent-sc33="" aria-label="Band" class="logo logo--portal logo-name--Band logo--image center" href="/"><img _ngcontent-sc33="" alt="Band" class="logo-image" height="35" src="https://pubimg.band.uol.com.br/Files/logo-band-2022-v2.png?v2" width="101"/><!-- --><!-- --><!-- --></a>

In [42]:
# Lista de links possivelmente de notícias no site da Band.
for i in n:
    try:
        print(i['href'])
    except:
        print("no link")

/
no link
https://bandplay.com/home
https://www.band.uol.com.br/noticias
https://www.band.uol.com.br/esportes
https://www.band.uol.com.br/entretenimento
https://receitas.band.uol.com.br/
https://www.band.uol.com.br/programacao
https://www.band.uol.com.br/band-shop
no link
no link
https://www.band.uol.com.br/ao-vivo
https://bandplay.com/home
https://www.band.uol.com.br/band-shop
https://www.band.uol.com.br/videos
https://www.band.uol.com.br/podcasts
https://www.band.uol.com.br/newsletters
https://www.band.uol.com.br/programacao
https://www.band.uol.com.br/show-da-fe
https://twitter.com/portaldaband
https://www.facebook.com/Band.com.br
https://www.youtube.com/c/BandBr/
https://www.instagram.com/bandtv/
https://www.band.uol.com.br/band-shop
no link
no link
https://www.band.uol.com.br/esportes/automobilismo/formula-1/noticias/f1-2023-espanha-16606363
https://www.band.uol.com.br/esportes/futebol/copa-do-brasil
https://www.band.uol.com.br/tudo-sobre/imposto-de-renda
https://www.band.uol.com.

## Folha de São Paulo 

Lendo feed RSS do site de notícias da [Folha de São Paulo](https://www1.folha.uol.com.br/feed/).

**Editoriais** Disponíveis

- [Em Cima Da Hora](https://feeds.folha.uol.com.br/emcimadahora/rss091.xml)
- [Opinião](https://feeds.folha.uol.com.br/opiniao/rss091.xml)
- [Política](https://feeds.folha.uol.com.br/poder/rss091.xml)
- [Mundo](https://feeds.folha.uol.com.br/mundo/rss091.xml)
- [Mercado](https://feeds.folha.uol.com.br/mercado/rss091.xml)
- [Cotidiano](https://feeds.folha.uol.com.br/cotidiano/rss091.xml)
- [Educação](https://feeds.folha.uol.com.br/educacao/rss091.xml)
- [Equilíbrio](https://feeds.folha.uol.com.br/equilibrio/rss091.xml)
- [Esporte](https://feeds.folha.uol.com.br/esporte/rss091.xml)
- [Ilustrada](https://feeds.folha.uol.com.br/ilustrada/rss091.xml)
- [Ilustríssima](https://feeds.folha.uol.com.br/ilustrissima/rss091.xml)
- [Ciência](https://feeds.folha.uol.com.br/ciencia/rss091.xml)
- [Ambiente](https://feeds.folha.uol.com.br/ambiente/rss091.xml)
- [Tec](https://feeds.folha.uol.com.br/tec/rss091.xml)
- [Comida](https://feeds.folha.uol.com.br/comida/rss091.xml)
- [Saúde](https://feeds.folha.uol.com.br/equilibrioesaude/rss091.xml)
- [Folhinha](https://feeds.folha.uol.com.br/folhinha/rss091.xml)
- [Turismo](https://feeds.folha.uol.com.br/turismo/rss091.xml)


In [43]:
import feedparser as fp

fsp_editoriais = [ 
    ('Em Cima Da Hora', 'https://feeds.folha.uol.com.br/emcimadahora/rss091.xml'),
    ('Opinião', 'https://feeds.folha.uol.com.br/opiniao/rss091.xml' ) ,
    ( 'Política', 'https://feeds.folha.uol.com.br/poder/rss091.xml') ,
    ('Mundo', 'https://feeds.folha.uol.com.br/mundo/rss091.xml'),
    ('Mercado','https://feeds.folha.uol.com.br/mercado/rss091.xml'),
    ('Cotidiano','https://feeds.folha.uol.com.br/cotidiano/rss091.xml'),
    ('Educação', 'https://feeds.folha.uol.com.br/educacao/rss091.xml'),
    ('Equilíbrio', 'https://feeds.folha.uol.com.br/equilibrio/rss091.xml'),
    ('Esporte','https://feeds.folha.uol.com.br/esporte/rss091.xml'),
    ('Ilustrada', 'https://feeds.folha.uol.com.br/ilustrada/rss091.xml'),
    ('Ilustríssima','https://feeds.folha.uol.com.br/ilustrissima/rss091.xml'),
    ('Ciência','https://feeds.folha.uol.com.br/ciencia/rss091.xml'),
    ('Ambiente','https://feeds.folha.uol.com.br/ambiente/rss091.xml'),
    ('Tec','https://feeds.folha.uol.com.br/tec/rss091.xml'),
    ('Comida','https://feeds.folha.uol.com.br/comida/rss091.xml'),
    ('Saúde','https://feeds.folha.uol.com.br/equilibrioesaude/rss091.xml'),
    ('Folhinha','https://feeds.folha.uol.com.br/folhinha/rss091.xml'),
    ('Turismo','https://feeds.folha.uol.com.br/turismo/rss091.xml'),
]

In [44]:
# Lista dos feeds pelos editoriais.
#l = [ {i: {'title': j.title, 'href': j.link } } for i,j in zip(range(len(d.entries) + 1), d.entries) 
#fsp_feeds = [ [ { i: { 'title': j.title, 'href': j.link } } for i, j in zip( range( len( fp.parser(feed) ) ), feed.entries ) ]  for feed in fsp_editoriais ]
fsp_feeds = [ 
    { i: ( j[0], fp.parse( j[1] ) ) }
    for i,j in zip( range( len( fsp_editoriais) + 1 ), fsp_editoriais ) 
]




In [45]:
fsp_feeds2 =  [ 
    { 'Editorial': j[0], 'FeedData': fp.parse( j[1] ) } 
    for j in fsp_editoriais  
]

In [46]:
# Lista de editoriais com suas respectivas listas de notícias. Para cada editorial extraido um lista de notícias.
fsp_news_list = [
    { 
        'Editorial': i[ 'Editorial' ],
        'Noticias': [ { 
            'Title': j.title,
            'Summary': j.summary,
            'href': j['link'].split('*')[1]
     }  for j in i[ 'FeedData' ].entries ]
    } for i in fsp_feeds2 
]

In [47]:
def trackFolhaSP():
    """
    Track página da "Folha de São Paulo"
    """
    fsp_editoriais = ( 
            ('Em Cima Da Hora', 'https://feeds.folha.uol.com.br/emcimadahora/rss091.xml'),
            ('Opinião', 'https://feeds.folha.uol.com.br/opiniao/rss091.xml' ) ,
            ( 'Política', 'https://feeds.folha.uol.com.br/poder/rss091.xml') ,
            ('Mundo', 'https://feeds.folha.uol.com.br/mundo/rss091.xml'),
            ('Mercado','https://feeds.folha.uol.com.br/mercado/rss091.xml'),
            ('Cotidiano','https://feeds.folha.uol.com.br/cotidiano/rss091.xml'),
            ('Educação', 'https://feeds.folha.uol.com.br/educacao/rss091.xml'),
            ('Equilíbrio', 'https://feeds.folha.uol.com.br/equilibrio/rss091.xml'),
            ('Esporte','https://feeds.folha.uol.com.br/esporte/rss091.xml'),
            ('Ilustrada', 'https://feeds.folha.uol.com.br/ilustrada/rss091.xml'),
            ('Ilustríssima','https://feeds.folha.uol.com.br/ilustrissima/rss091.xml'),
            ('Ciência','https://feeds.folha.uol.com.br/ciencia/rss091.xml'),
            ('Ambiente','https://feeds.folha.uol.com.br/ambiente/rss091.xml'),
            ('Tec','https://feeds.folha.uol.com.br/tec/rss091.xml'),
            ('Comida','https://feeds.folha.uol.com.br/comida/rss091.xml'),
            ('Saúde','https://feeds.folha.uol.com.br/equilibrioesaude/rss091.xml'),
            ('Folhinha','https://feeds.folha.uol.com.br/folhinha/rss091.xml'),
            ('Turismo','https://feeds.folha.uol.com.br/turismo/rss091.xml'),
        )

    fsp_feeds =  ( 
            { 'Editorial': j[0], 'FeedData': fp.parse( j[1] ) } 
                for j in fsp_editoriais  
        )  

    # Lista de editoriais com suas respectivas listas de notícias. Para cada editorial extraido um lista de notícias.
    fsp_news_list = (
            { 
                'Editorial': i[ 'Editorial' ],
                'Noticias': ( { 
                    'Title': j.title,
                    'Summary': j.summary,
                    'href': j['link'].split('*')[1]
            }  for j in i[ 'FeedData' ].entries )
            } for i in fsp_feeds 
        )

    return  fsp_news_list 

In [48]:
# Testes dos algoritmos

In [49]:
f = trackFolhaSP()


In [50]:
import sys

In [51]:

sys.getsizeof(f)

104

In [52]:
sys.getsizeof( fsp_news_list )

248

In [53]:
for i in f:
    print(i)

{'Editorial': 'Em Cima Da Hora', 'Noticias': <generator object trackFolhaSP.<locals>.<genexpr>.<genexpr> at 0x7f540e0b15b0>}
{'Editorial': 'Opinião', 'Noticias': <generator object trackFolhaSP.<locals>.<genexpr>.<genexpr> at 0x7f540e0b1460>}
{'Editorial': 'Política', 'Noticias': <generator object trackFolhaSP.<locals>.<genexpr>.<genexpr> at 0x7f540e0b3d10>}
{'Editorial': 'Mundo', 'Noticias': <generator object trackFolhaSP.<locals>.<genexpr>.<genexpr> at 0x7f540dfc4120>}
{'Editorial': 'Mercado', 'Noticias': <generator object trackFolhaSP.<locals>.<genexpr>.<genexpr> at 0x7f540e253df0>}
{'Editorial': 'Cotidiano', 'Noticias': <generator object trackFolhaSP.<locals>.<genexpr>.<genexpr> at 0x7f540dfc4a50>}
{'Editorial': 'Educação', 'Noticias': <generator object trackFolhaSP.<locals>.<genexpr>.<genexpr> at 0x7f540f72b060>}
{'Editorial': 'Equilíbrio', 'Noticias': <generator object trackFolhaSP.<locals>.<genexpr>.<genexpr> at 0x7f540e253df0>}
{'Editorial': 'Esporte', 'Noticias': <generator obj

In [54]:
for j in fsp_news_list:
    print(j)

{'Editorial': 'Em Cima Da Hora', 'Noticias': [{'Title': 'New Order e Blur agitam plateia grisalha em ótimos shows no Primavera Sound', 'Summary': '"Eu me sinto tão extraordinário", cantou Bernard Sumner, iniciando "True Faith", um dos vários clássicos do New Order no excelente show que o grupo apresentou nesta quinta (1º) no <a href="https://www1.folha.uol.com.br/folha-topicos/primavera-sound/" rel="" target="">Primavera Sound</a> Barcelona 2023.\n<a href="https://redir.folha.com.br/redir/online/emcimadahora/rss091/*https://www1.folha.uol.com.br/ilustrada/2023/06/new-order-e-blur-agitam-plateia-grisalha-em-otimos-shows-no-primavera-sound.shtml">Leia mais</a> (06/02/2023 - 14h35)', 'href': 'https://www1.folha.uol.com.br/ilustrada/2023/06/new-order-e-blur-agitam-plateia-grisalha-em-otimos-shows-no-primavera-sound.shtml'}, {'Title': "Ex-mulher que acusa tenista Thiago Wild de violência ironiza vitória no esporte: 'Virou melhor pessoa'", 'Summary': 'A influenciadora <a href="https://f5.fol

In [55]:
fsp_feeds[0][0]

('Em Cima Da Hora',
 {'bozo': True,
  'entries': [{'title': 'Troféu Audálio Dantas vai homenagear seis jornalistas em cerimônia em SP',
    'title_detail': {'type': 'text/plain',
     'language': None,
     'base': 'https://feeds.folha.uol.com.br/emcimadahora/rss091.xml',
     'value': 'Troféu Audálio Dantas vai homenagear seis jornalistas em cerimônia em SP'},
    'links': [{'rel': 'alternate',
      'type': 'text/html',
      'href': 'https://redir.folha.com.br/redir/online/emcimadahora/rss091/*https://www1.folha.uol.com.br/colunas/monicabergamo/2023/06/trofeu-audalio-dantas-vai-homenagear-seis-jornalistas-em-cerimonia-em-sp.shtml'}],
    'link': 'https://redir.folha.com.br/redir/online/emcimadahora/rss091/*https://www1.folha.uol.com.br/colunas/monicabergamo/2023/06/trofeu-audalio-dantas-vai-homenagear-seis-jornalistas-em-cerimonia-em-sp.shtml',
    'authors': [{}],
    'author': '',
    'summary': 'A quarta edição do Troféu Audálio Dantas vai homenagear seis jornalistas por seus tra

In [56]:
fsp_editoriais

[('Em Cima Da Hora', 'https://feeds.folha.uol.com.br/emcimadahora/rss091.xml'),
 ('Opinião', 'https://feeds.folha.uol.com.br/opiniao/rss091.xml'),
 ('Política', 'https://feeds.folha.uol.com.br/poder/rss091.xml'),
 ('Mundo', 'https://feeds.folha.uol.com.br/mundo/rss091.xml'),
 ('Mercado', 'https://feeds.folha.uol.com.br/mercado/rss091.xml'),
 ('Cotidiano', 'https://feeds.folha.uol.com.br/cotidiano/rss091.xml'),
 ('Educação', 'https://feeds.folha.uol.com.br/educacao/rss091.xml'),
 ('Equilíbrio', 'https://feeds.folha.uol.com.br/equilibrio/rss091.xml'),
 ('Esporte', 'https://feeds.folha.uol.com.br/esporte/rss091.xml'),
 ('Ilustrada', 'https://feeds.folha.uol.com.br/ilustrada/rss091.xml'),
 ('Ilustríssima', 'https://feeds.folha.uol.com.br/ilustrissima/rss091.xml'),
 ('Ciência', 'https://feeds.folha.uol.com.br/ciencia/rss091.xml'),
 ('Ambiente', 'https://feeds.folha.uol.com.br/ambiente/rss091.xml'),
 ('Tec', 'https://feeds.folha.uol.com.br/tec/rss091.xml'),
 ('Comida', 'https://feeds.folha.u

## Montando Lista de Notícias

In [57]:
number_news = 6


In [58]:
import random

In [59]:
news_list = []

news_list.extend( [ {'title': random.choices( [ (news['title'], news['url']) for news in dw ] , k = 2 ), 'source': 'G1/Globo' } ] )

news_list.extend( [ {'title': random.choices( [ (news['title'], news['href']) for news in cnn_list_news ] , k = 2 ), 'source': 'CNN Brasil' } ] )

news_list.extend( [ {'title': random.choices( [ (news['title'], news['href']) for news in bbc_news_lists ] , k = 2 ), 'source': 'BBC Brasil' } ] )

news_list.extend( [ { 'title':  random.choices( [ ( news['title'], news['href'] ) for news in band_news  ] , k= 2 ), 'source': 'Band' } ] )


In [60]:
for i in news_list:
    for j in i['title']:
        print(f"{j[0]}. \n{j[1]}\n")
    
    print(f"Fonte: {i['source']}\n\n")

Presidente do BC diz ser 'muito contra' moedas comuns entre países. 
https://g1.globo.com/economia/noticia/2023/06/02/presidente-do-banco-central-diz-ser-muito-contra-moedas-comuns-entre-paises.ghtml

Jeff Machado foi morto com fio de telefone, diz suspeito preso. 
https://g1.globo.com/rj/rio-de-janeiro/noticia/2023/06/02/garoto-de-programa-acusa-amigo-pela-morte-do-ator-jeff-e-confessa-que-vitima-foi-morta-com-fio-de-telefone-apos-ter-sido-dopado.ghtml

Fonte: G1/Globo


Pop. 
https://www.cnnbrasil.com.br/pop/

Saúde. 
https://www.cnnbrasil.com.br/saude/

Fonte: CNN Brasil


Por que críticos dizem que episódio final de Succession é 'perfeito' e 'brutal'. 
https://www.bbc.com/portuguese/articles/clj1883r823o

O que pode estar por trás da 'amnésia' de fãs de Taylor Swift após show da turnê Eras. 
https://www.bbc.com/portuguese/articles/c723xyk0pd7o

Fonte: BBC Brasil


Moraes ironiza Galo: "Quem mandou não saber bater pênalti". 
https://www.band.uol.com.br/minas-gerais/noticias/corinthi

## Salvando Todas Noticias (JSON)

### View news for test algorithm

In [61]:
# List news
news = None
portaisvar = [
    'dw', #Globo/G1
    'cnn_list_news', # CNN Brazil
    'bbc_news_list' # BBC Brazil
]

portais = { 'dw': 'Globo/G1', 'cnn_list_news': 'CNN Brasil', 'bbc_news_lists': 'BBC Brasil' }    

In [62]:
dw[0].keys(), cnn_list_news[0].keys(), bbc_news_lists[0].keys()

(dict_keys(['title', 'url']),
 dict_keys(['title', 'href']),
 dict_keys(['title', 'href']))

In [63]:
# Replace field name 'titulo' for 'title' name.
dw = [ { key.replace('titulo', 'title'):value for key, value in i.items() } for i in dw ]
dw = [ { key.replace('url', 'href'):value for key, value in i.items() } for i in dw ]
dw

[{'title': "VÍDEO: veja como foi a 1ª 'live' de Marte",
  'href': 'https://g1.globo.com/ciencia/noticia/2023/06/02/veja-como-foi-a-primeira-live-de-marte-feita-pela-agencia-espacial-europeia.ghtml'},
 {'title': 'Jeff Machado foi morto com fio de telefone, diz suspeito preso',
  'href': 'https://g1.globo.com/rj/rio-de-janeiro/noticia/2023/06/02/garoto-de-programa-acusa-amigo-pela-morte-do-ator-jeff-e-confessa-que-vitima-foi-morta-com-fio-de-telefone-apos-ter-sido-dopado.ghtml'},
 {'title': "Prisão no caso Jeff Machado, 'zona da morte' do Everest e mais vídeos",
  'href': 'https://g1.globo.com/playlist/videos-para-assistir-agora.ghtml'},
 {'title': "Dino visitou Lira após operação da PF: 'Falei o óbvio'",
  'href': 'https://g1.globo.com/politica/blog/andreia-sadi/post/2023/06/02/dino-e-lira-se-reunem-apos-operacao-policial-ordem-judicial-explica-ministro.ghtml'},
 {'title': "'Tem que falar com quem não gosta da gente', diz Lula sobre Congresso",
  'href': 'https://g1.globo.com/politica/n

In [64]:
#[ { key.replace('title', 'titulo' ) } ]
newslist = dw + cnn_list_news + bbc_news_lists
#newslist

In [65]:
import os
count = 0
try:
    os.mkdir( 'newsdata' )
except FileExistsError:
    print("File exist")
    
webdata = ''
# Save data of link news in files to folder.
for news in newslist:
    try:
        #news['data'] = requests.get(news['href']).text
        foldername = f'newsdata'
        #os.mkdir( foldername )
        news['datafile'] = f'{foldername}/'+str( count )
        with open( foldername+f'/{str(count)}', 'a' ) as fl:
            webdata = requests.get( news['href'] ).text 
            fl.write( webdata )
            fl.close()
            
        count += 1
    except :
        print('not get page')
        continue
    

File exist
not get page


In [66]:
newslist[0]['datafile']

'newsdata/0'

In [67]:
p = requests.get( newslist[0]['href'] )
soup = BeautifulSoup( p.text, 'html.parser')
texto = soup.get_text()
print( texto[0:150] )

 Veja como foi a primeira 'live' de Marte, feita pela Agência Espacial Europeia | Ciência | G1

                   Ciência                         





In [68]:
d = ''
with open(foldername+'/0', 'r') as fl:
    d = fl.read()
    fl.close()


soup = BeautifulSoup(d, 'html.parser')
#souptext = soup.text
content = soup.find_all('div')
#dir(soup)

In [69]:
texto = ' '.join( [ i.get_text() for i in content ] )
texto = texto.replace('\n', ' ')
with open('texto', 'w' ) as fl:
    #fl.write( souptext )
    fl.write( texto )
    

In [70]:
print( texto[0:100] )

              Goiás                                   fique por dentro             Imposto de Renda 


In [71]:
import json

with open('noticias.json', 'w') as fl:
    fl.write(json.dumps(newslist, ensure_ascii=False, indent=4))
    fl.close()

### Salva noticias

In [72]:
def savenews(newslist: list, filename: str) -> None:
    import json
    jsondata = None
    with open(filename, 'w') as fl:
        fl.write( json.dumps( newslist, encode='utf-8', indent=4 ) )
        fl.close()
    print('Saved...')
    

In [73]:
i = j = None

for i in news_list:
    for j in i['title']:
        print(f'{j[0]}.')
    print(f'Fonte: {i["source"] }\n\n ' )

Presidente do BC diz ser 'muito contra' moedas comuns entre países.
Jeff Machado foi morto com fio de telefone, diz suspeito preso.
Fonte: G1/Globo

 
Pop.
Saúde.
Fonte: CNN Brasil

 
Por que críticos dizem que episódio final de Succession é 'perfeito' e 'brutal'.
O que pode estar por trás da 'amnésia' de fãs de Taylor Swift após show da turnê Eras.
Fonte: BBC Brasil

 
Moraes ironiza Galo: "Quem mandou não saber bater pênalti".
Saiba onde e como assistir aos treinos e à corrida na Espanha.
Fonte: Band

 


In [74]:
i = j = None

print(f"Olá bem vindo ao Diário de Notícias Dimensão Alfa. Estas são as principais manchetes do dia.\n")
for i in news_list:
    print(f'Portal de Notícias {i["source"] }\n ' )
    for j in i['title']:
        print(f'{j[0]}.')

Olá bem vindo ao Diário de Notícias Dimensão Alfa. Estas são as principais manchetes do dia.

Portal de Notícias G1/Globo
 
Presidente do BC diz ser 'muito contra' moedas comuns entre países.
Jeff Machado foi morto com fio de telefone, diz suspeito preso.
Portal de Notícias CNN Brasil
 
Pop.
Saúde.
Portal de Notícias BBC Brasil
 
Por que críticos dizem que episódio final de Succession é 'perfeito' e 'brutal'.
O que pode estar por trás da 'amnésia' de fãs de Taylor Swift após show da turnê Eras.
Portal de Notícias Band
 
Moraes ironiza Galo: "Quem mandou não saber bater pênalti".
Saiba onde e como assistir aos treinos e à corrida na Espanha.


# Body Texts

# Sobre

## Dimensão Alfa

Dimensão Alfa projetos e conteúdos de tecnologia.

## Info

O presente projeto tem sido usado com fins de divulgação e facilitação de acesso a noticias e conhecimento em comunhão com objetivo da plataforma/página Dimensão Alfa. 
Conteúdos de terceiros são de responsabilidades dos mesmos bem como seus direitos autorais.

O projeto encontra-se em desenvolvimento, inicialmente fôra batizado de Ani Fátima Liu, e estará passando por alterações estando de inicio disponibilizado em formato "_jupyter notebook_" podendo servir como _case_ de estudo para os que se interessam por "web scrap" (raspagem de dados).

Tecnologias foram usadas para gerar vídeo de noticias diária para página [Youtube](https://www.youtube.com/@dimensaoalfa); foi usada as seguintes tecnologias:

* [Editor de códigos VSCode](https://code.visualstudio.com/)
* [Python (linguagem de programação)](https://www.python.org/)
* [Ambiente JupyterLab](https://jupyter.org/)
* [Biblioteca "Requests"](https://requests.readthedocs.io/en/latest/)
* [Biblioteca "BeautifulSoup"](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
* [Serviço de Sintese de Voz Microsoft/Azure](https://speech.microsoft.com)


Peço e agradeço a compreensão e apoio de todos. 

Para contribuições, dúvidas, sugestões visitem meu blog [WSRicardo](https://wsricardo.blogspot.com).


## Sugestões de Conteúdo

Deixamos abaixo algumas sugestões de conteúdos e canais com recursos para estudos e pesquisa que podem ser uteis para quem se interessa por tecnologia, programação de computadores, matemática, ciências de dados e inteligência artificial.

* [Programação Dinâmica](https://www.youtube.com/c/Programa%C3%A7%C3%A3oDin%C3%A2mica)
* [Toda Matemática](https://www.youtube.com/c/GustavoViegascurso)
* [Matemática Universitária](https://www.youtube.com/c/Matem%C3%A1ticaUniversit%C3%A1riaProfRenan)
* [Reflexões Matemáticas](https://www.youtube.com/c/Reflex%C3%B5esMatem%C3%A1ticasDrDilbertoJ%C3%BAnior)
* [Programação Descomplicada](https://www.youtube.com/user/progdescomplicada)
* [Univesp](https://www.youtube.com/user/univesptv)
* [USP no Youtube](https://www.youtube.com/c/CanalUSP)
* [IME/USP](https://www.ime.usp.br/)
* [IMPA](https://www.youtube.com/c/impabr)



## Links

* [Dimensão Alfa](https://www.dimensaoalfa.com.br)
* [Facebook](https://www.facebook.com/dimensaoalfa)
* [Youtube](https://www.youtube.com/@dimensaoalfa)
* [WSRicardo](https://wsricardo.blogspot.com)